In [ ]:
!pwd
!ls

In [ ]:
path = "/content/drive/MyDrive/ELEC301"
os.chdir(path)

In [ ]:
!unzip 'data.zip'

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile

In [ ]:
path = "/content/drive/MyDrive/ELEC301/data"
os.chdir(path)

In [ ]:
# Time Domain Data
data_t = []
label = []
for filename in os.listdir():
    fs, data = wavfile.read(filename)
    if len(data.shape) > 1:
        data = data[:, 0]
    data = data.astype(float)
    data_t.append(data)
    label.append(str(filename[: -7]))

In [ ]:
threshold = 0.1
cut = []
for i in range(1125):
  effect = data_t[i][np.argwhere(data_t[i]/max(data_t[i]) > threshold)[0,0]:np.argwhere(data_t[i]/max(data_t[i]) > threshold)[-1,0]+1]
  cut.append(effect)


In [ ]:
# Put Data Into 2D Matrix
D = []
maxl = 0
for i in range(1125):
  print(len(cut[i]))
  seg_data = []
  for j in range(10):
    data = cut[i]
    seg = int(len(data)/(10))
    if j < (9):
      seg_data.append(data[j*seg:j*seg+seg])
    else:
      seg_data.append(data[9*seg:-1])
  D.append(seg_data)

In [ ]:
D = np.array(D)

In [ ]:
for i in range(1125):
  for j in range(10):
    if len(D[i,j]) > maxl:
      maxl = len(D[i,j])

In [ ]:
min

In [ ]:
data_pad = [] # Time Domain Data
for i in range(1125):
  zero_pad = np.zeros((10, maxl))
  for j in range(10):
    zero_pad[j, range(0, len(D[i,j]))] = D[i,j]
    #zero_pad[j] = D[i,j][range(min)]
  data_pad.append(zero_pad)

In [ ]:
data_pad = np.array(data_pad)
data_pad.shape

In [ ]:
import librosa
from librosa import display

In [ ]:
# MFCC
MFCC = np.zeros((1125,96,32,10))
for i in range(1125):
  for j in range(10):
    MFCC[i,:,:,j] = (librosa.feature.mfcc(y=data_pad[i,j], sr=fs, n_mfcc = 32*3))

In [ ]:
# MFCC_11250
MFCC = []
for i in range(1125):
  mid = []
  print(i)
  for j in range(10):
    mid.append(librosa.feature.mfcc(y=D[i,j], sr=fs, n_mfcc = 32))
  MFCC.append(mid)

In [ ]:
# Chroma
Chroma = np.zeros((1125,96,32,10))
for i in range(1125):
  for j in range(10):
    Chroma[i,:,:,j] = (librosa.feature.chroma_stft(y=data_pad[i,j], sr=fs, n_chroma = 32*3))
  print(i)
Chroma = np.array(Chroma)

In [ ]:
# Chroma_11250
Chroma = []
for i in range(1125):
  mid=[]
  for j in range(10):
    mid.append(librosa.feature.chroma_stft(y=data_pad[i,j], sr=fs, n_chroma = 32))
  Chroma.append(mid)
  print(i)

In [ ]:
# Mel Spectrogram
MELS = np.zeros((1125,96,32,10))
for i in range(1125):
  for j in range(10):
    MELS[i,:,:,j] = (librosa.feature.melspectrogram(y=data_pad[i,j], sr=fs, n_mels = 32*3))
  MELS = np.array(MELS)

In [ ]:
# Mel Spectrogram_11250
MELS = []
for i in range(1125):
  print(i)
  mid = []
  for j in range(10):
    mid.append(librosa.feature.melspectrogram(y=data_pad[i,j], sr=fs, n_mels = 32))
  MELS.append(mid)

In [ ]:
print(MELS.shape)
print(Chroma.shape)
print(MFCC.shape)

(1125, 96, 32, 10)
(1125, 96, 32, 10)
(1125, 96, 32, 10)


In [ ]:
type(MFCC[0][0])

In [ ]:
RED=np.zeros((1125,10,32,6))
for i in range(1125):
  for j in range(10):
    mid1=[]
    mid2=[]
    mid3=[]
    ave = np.mean(MFCC[i][j],axis=1)
    vari = np.var(MFCC[i][j],axis=1)
    mid1.append(ave)
    mid1.append(vari)
    ave = np.mean(Chroma[i][j],axis=1)
    vari = np.var(Chroma[i][j],axis=1)
    mid2.append(ave)
    mid2.append(vari)
    ave = np.mean(MELS[i][j],axis=1)
    vari = np.var(MELS[i][j],axis=1)
    mid3.append(ave)
    mid3.append(vari)
    mid1=np.array(mid1).transpose()
    mid2=np.array(mid2).transpose()
    mid3=np.array(mid3).transpose()
    val=np.column_stack([mid1,mid2,mid3])
    RED[i,j] = val


In [ ]:
DATA_M = []
for i in range(1125):
  for j in range(10):
    mat = np.column_stack([MFCC[i],Chroma[i],MELS[i]])
    DATA_M.append(mat)
DATA_M=np.array(DATA_M)

In [ ]:
DATA_M = np.zeros((11250, 12, 10, 3))
for i in range(11250):
  DATA_M[i,:,:,0] = MFCC[i]
  DATA_M[i,:,:,1] = Chroma[i]
  DATA_M[i,:,:,2] = MELS[i]

In [ ]:
DATA_M.shape

(11250, 96, 96)

In [ ]:
"""
import matplotlib.pyplot as plt
fig, ax = plt.subplots(nrows=2, sharex=True)
img = librosa.display.specshow(res,
                               x_axis='time', y_axis='mel', fmax=8000,
                               ax=ax[0])
fig.colorbar(img, ax=[ax[0]])
ax[0].set(title='Mel spectrogram')
ax[0].label_outer()
img = librosa.display.specshow(res2, x_axis='time', ax=ax[1])
fig.colorbar(img, ax=[ax[1]])
ax[1].set(title='MFCC')
"""

In [ ]:
"""
from python_speech_features import mfcc
MFCC = []
for i in range(1125):
  mid = []
  for j in range(6):
    features_mfcc = mfcc(D[i][j], fs, nfft=1200)
    features_mfcc = features_mfcc[:, 1:]
    features_mfcc = features_mfcc.T
    mfcc_processed = np.column_stack([np.mean(features_mfcc,axis=1), np.var(features_mfcc,axis=1)])
    mid.append(mfcc_processed)
  MFCC.append(mid)
"""

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display

In [ ]:
"""
def get_spectrogram(waveform):
    #spectrogram = tf.signal.stft(waveform, frame_length=512, frame_step=128)
    spectrogram = tf.signal.stft(waveform, frame_length=128, frame_step=512)
    spectrogram = tf.abs(spectrogram)
    spectrogram = spectrogram[..., tf.newaxis]
    return spectrogram
"""

In [ ]:
"""
SPEC = []
for i in range(1125):
  print(i)
  mid = []
  for j in range(6):
    waveform = data_pad[i][j]
    spectrogram = get_spectrogram(waveform)
    mid.append(spectrogram)
  SPEC.append(mid)

spectrogram.shape
  #print('Label:', label_name)
  #print('Waveform shape:', waveform.shape)
  #print('Spectrogram shape:', spectrogram.shape)
  #print('Audio playback')
  #display.display(display.Audio(waveform, rate=48000))
"""

In [ ]:
"""
DATA = np.zeros((1125,53,65,6))
for i in range(1125):
  for j in range(53):
    for k in range(65):
      for m in range(6):
        DATA[i,j,k,m] = SPEC[i,m,j,k]
"""

In [ ]:
# Make Number Label (1125)
Y = []
for emo in range(0, len(label)):
    if label[emo] == 'angry':
        Y.append(0)
    if label[emo] == 'calm':
        Y.append(1)
    if label[emo] == 'disgust':
        Y.append(2)
    if label[emo] == 'fearful':
        Y.append(3)
    if label[emo] == 'happy':
        Y.append(4)
    if label[emo] == 'neutral':
        Y.append(5)
    if label[emo] == 'sad':
        Y.append(6)
    if label[emo] == 'surprised':
        Y.append(7)
Y = np.array(Y)

In [ ]:
# Make Number Label
Y = []
for emo in range(0, len(label)):
    if label[emo] == 'angry':
      for i in range(10):
        Y.append(0)
    if label[emo] == 'calm':
      for i in range(10):
        Y.append(1)
    if label[emo] == 'disgust':
      for i in range(10):
        Y.append(2)
    if label[emo] == 'fearful':
      for i in range(10):
        Y.append(3)
    if label[emo] == 'happy':
      for i in range(10):
        Y.append(4)
    if label[emo] == 'neutral':
      for i in range(10):
        Y.append(5)
    if label[emo] == 'sad':
      for i in range(10):
        Y.append(6)
    if label[emo] == 'surprised':
      for i in range(10):
        Y.append(7)
Y = np.array(Y)

In [ ]:
DATA = np.zeros((11250,96,96,1))
for i in range(11250):
  print(i)
  for j in range(96):
    for k in range(96):
      DATA[i,j,k,0] = DATA_M[i,j,k]

In [ ]:
DATA.shape

In [ ]:
# Select Training and Test Data
random_traning_data = np.random.choice(len(Y), 2000, replace=False)
train_data = DATA[random_traning_data]
train_label = Y[random_traning_data]
remaining_data = np.setdiff1d(range(0, len(Y)), random_traning_data)
test_data = DATA[remaining_data]
test_label = Y[remaining_data]
len(test_label)+len(train_label)

In [ ]:
input_shape = DATA.shape[1:]
num_labels = 8
# Instantiate the `tf.keras.layers.Normalization` layer.
norm_layer = layers.Normalization()
norm_layer.adapt(train_data)

model = models.Sequential([
    layers.Input(shape=input_shape),

    #Downsample the input.
    #layers.Resizing(10, 10),

    norm_layer,
    layers.Conv2D(32, 5, activation='softmax'),
    #layers.MaxPooling2D(),
    layers.Conv2D(32, 5, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 5, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 5, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 5, activation='relu'),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_labels),
])
model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])
history = model.fit(
    train_data, train_label, epochs=100, validation_data=(test_data, test_label))


Epoch 1/100
63/63 [==============================] - 170s 3s/step - loss: 2.0704 - accuracy: 0.1330 - val_loss: 2.0147 - val_accuracy: 0.1342
Epoch 2/100
63/63 [==============================] - 164s 3s/step - loss: 2.0378 - accuracy: 0.1590 - val_loss: 1.9837 - val_accuracy: 0.1734
Epoch 3/100
63/63 [==============================] - 163s 3s/step - loss: 1.9982 - accuracy: 0.1745 - val_loss: 1.9583 - val_accuracy: 0.1836
Epoch 4/100
63/63 [==============================] - 166s 3s/step - loss: 1.9737 - accuracy: 0.1940 - val_loss: 1.9487 - val_accuracy: 0.1808
Epoch 5/100
63/63 [==============================] - 166s 3s/step - loss: 1.9525 - accuracy: 0.2145 - val_loss: 1.9102 - val_accuracy: 0.2579
Epoch 6/100
63/63 [==============================] - 166s 3s/step - loss: 1.9088 - accuracy: 0.2085 - val_loss: 1.9112 - val_accuracy: 0.2411
Epoch 7/100
63/63 [==============================] - 165s 3s/step - loss: 1.9097 - accuracy: 0.2200 - val_loss: 1.8734 - val_accuracy: 0.2577
Epoch 

KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
test_loss, test_acc = model.evaluate(test_data,  test_label, verbose=2)
print(test_acc)

PREDICTION

In [ ]:
path = "/content/drive/MyDrive/ELEC301/test"
os.chdir(path)

In [ ]:
# Time Domain Data
data_test = []
label_test = []
NUM = []
L_test = [] # length of each audio
for filename in os.listdir():
    label_test.append(str(filename[: -7]))
    fs, data = wavfile.read(filename)
    if len(data.shape) > 1:
        data = data[:, 0]
    data = data.astype(float)
    data_test.append(data)
    L_test.append(len(data))
    NUM.append(str(filename[6: 9]))
print(L_test)

In [ ]:
for i in range(315):
  data_test[i] = data_test[i].astype(float)

In [ ]:
data_test[0][148197]

5823.0

In [ ]:
cut = []
for i in range(315):
  posi = np.argmax(data_test[i])
  small = np.argwhere(data_test[i]/data_test[i][posi] > 0.1)[0,0]
  big = np.argwhere(data_test[i]/data_test[i][posi] > 0.1)[-1,0]
  eff = data_test[i][small : big+1]
  cut.append(eff)

In [ ]:
# Put Data Into 2D Matrix
D_test = []
for i in range(315):
  seg_data = []
  for j in range(6):
    data = cut[i]
    seg = int(len(data)/(6))
    if j < (5):
      seg_data.append(data[j*seg:j*seg+seg])
    else:
      seg_data.append(data[5*seg:-1])
  D_test.append(seg_data)

In [ ]:
D_test = np.array(D_test)

In [ ]:
D_test.shape

In [ ]:
max = 0
for i in range(315):
  for j in range(6):
    if len(D_test[i,j]) > max:
      max = len(D_test[i,j])

In [ ]:
data_pad1 = [] # Time Domain Data
for i in range(315):
  zero_pad = np.zeros((6, 27162))
  for j in range(6):
    zero_pad[j, range(0, len(D_test[i,j]))] = D_test[i,j]
  data_pad1.append(zero_pad)

In [ ]:
data_pad1 = np.array(data_pad1)
data_pad1.shape

(315, 6, 27162)

In [ ]:
# MFCC
MFCC_test = []
for i in range(315):
  mid = []
  for j in range(6):
    features_mfcc = mfcc(data_pad1[i][j], fs, nfft=1200)
    features_mfcc = features_mfcc[:, 1:]
    features_mfcc = features_mfcc.T
    mfcc_processed = np.column_stack([np.mean(features_mfcc,axis=1), np.var(features_mfcc,axis=1)])
    mid.append(mfcc_processed)
  MFCC_test.append(mid)

In [ ]:
MFCC_test = np.array(MFCC_test)
MFCC_test.shape

(315, 6, 12, 2)

In [ ]:
predictions = model.predict(MFCC_test)
score = tf.nn.softmax(predictions)
label_predict = []
for i in range(315):
  label_predict.append(np.argmax(score[i]))

label_predict

In [ ]:
res = []
for emo in range(0, len(label_predict)):
    if label_predict[emo] == 0:
      res.append('angry')
    if label_predict[emo] == 1:
      res.append('calm')
    if label_predict[emo] == 2:
      res.append('disgust')
    if label_predict[emo] == 3:
      res.append('fearful')
    if label_predict[emo] == 4:
      res.append('happy')
    if label_predict[emo] == 5:
      res.append('neutral')
    if label_predict[emo] == 6:
      res.append('sad')
    if label_predict[emo] == 7:
      res.append('surprised')
print(res)

In [ ]:
res = np.array(res).transpose()
NUM = np.array(NUM).transpose()
sample = np.column_stack([NUM, res])
sort_sample = sample[sample[:, 0].argsort()]
name = []
for item in sort_sample[:,0]:
  item = 'sample'+str(item)
  name.append(item)
name = np.array(name).transpose()
sort_sample[:,0] = name
sort_sample

In [ ]:
import pandas as pd
DF = pd.DataFrame(sort_sample)
DF.to_csv("res.csv")

RNN BELOW

In [ ]:
batch_size = 192
# Each MNIST image batch is a tensor of shape (batch_size, 28, 28).
# Each input sequence will be of size (28, 28) (height is treated like time).
input_dim = 96

units = 192
output_size = 8  # labels are from 0 to 7
def build_model(allow_cudnn_kernel=True):
    # CuDNN is only available at the layer level, and not at the cell level.
    # This means `LSTM(units)` will use the CuDNN kernel,
    # while RNN(LSTMCell(units)) will run on non-CuDNN kernel.
    if allow_cudnn_kernel:
        # The LSTM layer with default options uses CuDNN.
        lstm_layer = layers.LSTM(units, input_shape=(None, input_dim))
    else:
        # Wrapping a LSTMCell in a RNN layer will not use CuDNN.
        lstm_layer = layers.RNN(
            layers.LSTMCell(units), input_shape=(None, input_dim)
        )
    model = models.Sequential(
        [
            lstm_layer,
            layers.BatchNormalization(),
            layers.Dense(output_size),
        ]
    )
    return model

In [ ]:
model = build_model(allow_cudnn_kernel=True)

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="sgd",
    metrics=["accuracy"],
)
history = model.fit(
    train_data, train_label, epochs=100, batch_size=batch_size, validation_data=(test_data, test_label))

In [ ]:
noncudnn_model = build_model(allow_cudnn_kernel=False)
noncudnn_model.set_weights(model.get_weights())
noncudnn_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="sgd",
    metrics=["accuracy"],
)
history = model.fit(
    train_data, train_label, epochs=100, batch_size=batch_size, validation_data=(test_data, test_label))

Epoch 1/100
11/11 [==============================] - 7s 640ms/step - loss: 2.0215 - accuracy: 0.2435 - val_loss: 2.0507 - val_accuracy: 0.2218
Epoch 2/100
11/11 [==============================] - 7s 627ms/step - loss: 1.9734 - accuracy: 0.2645 - val_loss: 2.0365 - val_accuracy: 0.2396
Epoch 3/100
11/11 [==============================] - 7s 628ms/step - loss: 1.9278 - accuracy: 0.2880 - val_loss: 2.0116 - val_accuracy: 0.2409
Epoch 4/100
11/11 [==============================] - 7s 620ms/step - loss: 1.9080 - accuracy: 0.2875 - val_loss: 1.9922 - val_accuracy: 0.2493
Epoch 5/100
11/11 [==============================] - 7s 636ms/step - loss: 1.9077 - accuracy: 0.2795 - val_loss: 1.9943 - val_accuracy: 0.2427
Epoch 6/100
11/11 [==============================] - 7s 614ms/step - loss: 1.8871 - accuracy: 0.2840 - val_loss: 1.9819 - val_accuracy: 0.2422
Epoch 7/100
11/11 [==============================] - 7s 617ms/step - loss: 1.8744 - accuracy: 0.2965 - val_loss: 1.9951 - val_accuracy: 0.2511

KeyboardInterrupt: ignored